In [15]:
import numpy as np
import pandas as pd


### Loading and Preprocessing of training data

In [16]:
train=pd.read_excel("Data_Train (1).xlsx")

In [24]:
test=pd.read_excel('Data_Test (2).xlsx')

In [9]:
train.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [10]:
test.head()

,STORY
0,2019 will see gadgets like gaming smartphones ...
1,It has also unleashed a wave of changes in the...
2,It can be confusing to pick the right smartpho...
3,The mobile application is integrated with a da...
4,We have rounded up some of the gadgets that sh...


In [0]:
y=train.SECTION

In [0]:
story=train.STORY

In [13]:
story.shape

(7628,)

In [21]:

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [22]:
import nltk
nltk.download('stopwords')
sw=set(stopwords.words('english'))
ps=PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sachin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
len(story[1])

129

In [0]:
s=story.values

In [18]:
def clean_data(data):
    cleaned_data=[]
    for i in data:
        #i=i.split('\n')
        i=i.split()
        a=[s.lower() for s in i if s not in sw]
        #c=[ps.stem(ix) for ix in a]
        c=' '.join(a)
        cleaned_data.append(c)
    return cleaned_data

In [0]:
cleaned_data=clean_data(s)

In [20]:
len(cleaned_data[1])

118

# LSTM


In [0]:
def readGloveFile(gloveFile):
    with open(gloveFile, 'r',encoding='utf-8') as f:
        wordToGlove = {}  # map from a token (word) to a Glove embedding vector
        wordToIndex = {}  # map from a token to an index
        indexToWord = {}  # map from an index to a token 

        for line in f:
            record = line.strip().split()
            token = record[0] 
            wordToGlove[token] = np.array(record[1:], dtype=np.float64)

        tokens = sorted(wordToGlove.keys())
        for idx, tok in enumerate(tokens):
            kerasIdx = idx + 1 
            wordToIndex[tok] = kerasIdx 
            indexToWord[kerasIdx] = tok 

    return wordToIndex, indexToWord, wordToGlove

# Creating Pretrained Keras Embedding Layer
def createPretrainedEmbeddingLayer(wordToGlove, wordToIndex, isTrainable):
    vocabLen = len(wordToIndex) + 1  
    embDim = next(iter(wordToGlove.values())).shape[0] 

    embeddingMatrix = np.zeros((vocabLen, embDim)) 
    for word, index in wordToIndex.items():
        embeddingMatrix[index, :] = wordToGlove[word]

    embeddingLayer = Embedding(vocabLen, embDim, weights=[embeddingMatrix], trainable=isTrainable)
    return embeddingLayer




In [22]:
from keras.layers import *
wordToIndex, indexToWord, wordToGlove = readGloveFile("/content/drive/My Drive/glove.6B.50d.txt")
pretrainedEmbeddingLayer = createPretrainedEmbeddingLayer(wordToGlove, wordToIndex, False)

W0811 06:48:37.483144 140642066630528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [0]:
from keras.preprocessing import sequence
padded_list=[]
for i in cleaned_data:
    i =i.split()
    temp_list=[]
    for j in i:
        if wordToIndex.get(j):
            temp_list.append(wordToIndex[j])
        else:
            temp_list.append(0)
    
    padded_seq=sequence.pad_sequences([temp_list],200,padding='post')
    #print(padded_seq)
    padded_list.append(padded_seq)
    
padded_list=np.array(padded_list)

In [24]:
print(padded_list[500])

[[ 84028      0 286468  92463 116894  49495  91556      0 243977 260060
  137982 235545 101291      0  45893 146354 198478 354461 302352 338995
  359889      0 185457 146662 222471 185457 143219 385186      0  84028
  315709 142428 239464 191138 293387 362970      0 338276 211025  84028
  217594 134888 386458  71587 277915 167631 188837 271493      0 192758
  232536      0 128557      0  91556  53499      0      0 183773      0
       0 358307      0      0  77486 387210      0      0 174783      0
       0 123789 352233 320243  91556 235545 239798 347914 148908 359889
   63946 137065      0  84028      0      0 357959      0      0 225985
  318864 167631      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0

In [25]:
padded_list.shape

(7628, 1, 200)

In [0]:
padded_list=padded_list.reshape(7628,200)

In [27]:

padded_list.shape


(7628, 200)

In [0]:
from keras.utils import np_utils

In [0]:
y_categorical=np_utils.to_categorical(y)

In [0]:
from keras.layers import *
from keras.models import Sequential

In [31]:
model=Sequential()
model.add(pretrainedEmbeddingLayer)
model.add(LSTM(128,activation='tanh',dropout=0.3,return_sequences=True))
model.add(LSTM(64,activation='tanh',dropout=0.2))

model.add(Dense(4,activation='softmax'))
model.summary()

W0811 06:49:12.749156 140642066630528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 06:49:12.775582 140642066630528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0811 06:49:12.788398 140642066630528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0811 06:49:12.789323 140642066630528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0811 06:49:16.351055 140642066630528 depre

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          20000050  
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         91648     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 20,141,366
Trainable params: 141,316
Non-trainable params: 20,000,050
_________________________________________________________________


In [0]:
from keras.callbacks import ModelCheckpoint
checkpoint =ModelCheckpoint('best_model_news1.h5', monitor='acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [33]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

W0811 06:49:26.001060 140642066630528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [36]:
hist=model.fit(padded_list,y_categorical,epochs=20,batch_size=64,callbacks=[checkpoint])


W0811 06:49:57.463284 140642066630528 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
 512/7628 [=>............................] - ETA: 1:22 - loss: 1.3696 - acc: 0.3555

KeyboardInterrupt: ignored

In [1]:

from keras.models import load_model

Using TensorFlow backend.


In [2]:
m=load_model('best_model_news.h5')

W0813 23:11:57.472580 11376 deprecation_wrapper.py:119] From c:\users\sachin\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0813 23:11:58.503024 11376 deprecation_wrapper.py:119] From c:\users\sachin\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0813 23:11:58.662842 11376 deprecation_wrapper.py:119] From c:\users\sachin\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0813 23:11:59.198842 11376 deprecation_wrapper.py:119] From c:\users\sachin\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default 

# Test data Preprocessing for LSTM

In [25]:
test=test.STORY
clean_test=clean_data(test)

In [26]:
word_test_list=[]
for w in clean_test:
    w=w.split()
    for j in w:
        word_test_list.append(j)

In [27]:
word_test_list=np.array(word_test_list)
uniq_test_list=np.unique(word_test_list)


In [28]:
word_2index_test={val:key for key,val in enumerate(uniq_test_list)}


In [29]:
from keras.preprocessing import sequence
padded_test_list=[]
for i in clean_test:
    i =i.split()
    temp_list=[]
    for j in i:
        if wordToIndex.get(j):
            temp_list.append(wordToIndex[j])
        else:
            temp_list.append(0)

    padded_seq=sequence.pad_sequences([temp_list],200,padding='post')
    #print(padded_seq)
    padded_test_list.append(padded_seq)
    
padded_test_list=np.array(padded_test_list)

NameError: name 'wordToIndex' is not defined

In [45]:
padded_test_list.shape

(2748, 1, 200)

In [0]:
padded_test_list=padded_test_list.reshape(2748,200)

In [0]:
padded_test_list

In [0]:
p=np.array(padded_test_list)

In [0]:
pred=m.predict(p)

In [56]:
pred

array([[1.9901151e-04, 9.9755424e-01, 8.0940779e-05, 2.1657832e-03],
       [1.3310500e-02, 1.1960854e-01, 8.6666006e-01, 4.2088071e-04],
       [1.9196443e-04, 9.9884713e-01, 6.8195783e-05, 8.9269161e-04],
       ...,
       [2.3754808e-04, 9.9911255e-01, 6.9827562e-05, 5.8008533e-04],
       [9.8962003e-01, 3.9508389e-03, 1.2785339e-04, 6.3012294e-03],
       [6.6255889e-04, 9.9595863e-01, 8.6823995e-05, 3.2920367e-03]],
      dtype=float32)

In [0]:
prediction_test_list=[]
for i in pred:
    m=np.argmax(i)
    prediction_test_list.append(m)

In [0]:
submit1=pd.DataFrame({'SECTION':prediction_test_list})
submit1.to_csv('new_category_pred_LSTM2.csv',index=False)